# 🎓 Student Performance Prediction Project
---
### Algorithms: Decision Tree, SVM, and Hybrid CNN+SVM
**Dataset:** `student_data.csv`

This notebook predicts student performance levels (Low / Medium / High) using multiple ML and DL models.

In [1]:

# STEP 1️⃣: Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore')


In [2]:

# STEP 2️⃣: Load Dataset
data = pd.read_csv("student_data.csv")
print("✅ Dataset Loaded Successfully!")
print("Shape:", data.shape)
print("\nPreview of Data:")
display(data.head())


✅ Dataset Loaded Successfully!
Shape: (240, 9)

Preview of Data:


,Math_Score,Science_Score,English_Score,Attendance,Assignments_Submitted,Hours_Study,Parent_Education_Level,Extra_Curricular,Performance
0,78,76,31,83,7,5,2,1,1
1,91,78,55,82,13,4,1,0,2
2,68,58,46,91,11,5,0,0,0
3,54,49,69,96,11,4,0,0,0
4,82,93,62,71,10,5,2,0,2


In [3]:

# STEP 3️⃣: Split Features and Target
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

print("Training samples:", X_train.shape[0])
print("Testing samples:", X_test.shape[0])


Training samples: 180
Testing samples: 60


In [4]:

# STEP 4️⃣: Decision Tree Classifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_test)

print("\n📊 Decision Tree Results:")
print("Accuracy:", round(accuracy_score(y_test, dt_pred), 4))
print("Precision:", round(precision_score(y_test, dt_pred, average='macro'), 4))
print("Recall:", round(recall_score(y_test, dt_pred, average='macro'), 4))
print("F1 Score:", round(f1_score(y_test, dt_pred, average='macro'), 4))



📊 Decision Tree Results:
Accuracy: 0.7
Precision: 0.6985
Recall: 0.7139
F1 Score: 0.6995


In [5]:

# STEP 5️⃣: Support Vector Machine (SVM)
svm = SVC(kernel='rbf', random_state=42)
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_test)

print("\n📊 SVM Results:")
print("Accuracy:", round(accuracy_score(y_test, svm_pred), 4))
print("Precision:", round(precision_score(y_test, svm_pred, average='macro'), 4))
print("Recall:", round(recall_score(y_test, svm_pred, average='macro'), 4))
print("F1 Score:", round(f1_score(y_test, svm_pred, average='macro'), 4))



📊 SVM Results:
Accuracy: 0.9167
Precision: 0.9183
Recall: 0.9268
F1 Score: 0.9194


In [6]:

# STEP 6️⃣: Hybrid CNN + SVM
# Prepare 3D data for CNN
X_train_cnn = np.expand_dims(X_train, axis=2)
X_test_cnn = np.expand_dims(X_test, axis=2)

cnn = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)),
    MaxPooling1D(2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # assuming 3 output classes
])

cnn.compile(optimizer=Adam(learning_rate=0.001),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

print("\n⏳ Training CNN Model...")
cnn.fit(X_train_cnn, y_train, epochs=10, batch_size=16, verbose=1)

# Extract CNN features (second last layer output)
extract_model = Sequential(cnn.layers[:-1])
cnn_features_train = extract_model.predict(X_train_cnn)
cnn_features_test = extract_model.predict(X_test_cnn)

# Train SVM on CNN features
hybrid_svm = SVC(kernel='rbf', random_state=42)
hybrid_svm.fit(cnn_features_train, y_train)
hybrid_pred = hybrid_svm.predict(cnn_features_test)

print("\n📊 Hybrid CNN + SVM Results:")
print("Accuracy:", round(accuracy_score(y_test, hybrid_pred), 4))
print("Precision:", round(precision_score(y_test, hybrid_pred, average='macro'), 4))
print("Recall:", round(recall_score(y_test, hybrid_pred, average='macro'), 4))
print("F1 Score:", round(f1_score(y_test, hybrid_pred, average='macro'), 4))

print("\n✅ All models executed successfully!")



⏳ Training CNN Model...
Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 46ms/step - accuracy: 0.3111 - loss: 9.3229 
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.3611 - loss: 5.2935
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.3667 - loss: 2.7442
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4889 - loss: 1.4233
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.4722 - loss: 1.1450
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.4556 - loss: 1.0282
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4722 - loss: 0.9877
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.5444 - loss: 0.9218
Epoch 9/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.4500 - loss: 0.9517
Epoch 10/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.5111 - loss: 0.9521
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step

📊 Hybrid CNN + SVM Results:
Accuracy: 0.5333
Pr